## Implementing binary decision trees

The goal of this notebook is to implement your own binary decision tree classifier. You will:
    
* Transform categorical variables into binary variables.
* Write a function to compute the number of misclassified examples in an intermediate node.
* Write a function to find the best feature to split on.
* Build a binary decision tree from scratch.
* Make predictions using the decision tree.
* Evaluate the accuracy of the decision tree.
* Visualize the decision at the root node.

**Important Note**: In this assignment, we will focus on building decision trees where the data contain **only binary (0 or 1) features**. This allows us to avoid dealing with:
* Multiple intermediate nodes in a split
* The thresholding issues of real-valued features.

# Load the lending club dataset

In [1]:
# Import libraries

import pandas as pd
import numpy as np

In [2]:
loans = pd.read_csv('lending-club-data.csv')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (19,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
loans.head(5)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,sub_grade_num,delinq_2yrs_zero,pub_rec_zero,collections_12_mths_zero,short_emp,payment_inc_ratio,final_d,last_delinq_none,last_record_none,last_major_derog_none
0,1077501,1296599,5000,5000,4975,36 months,10.65,162.87,B,B2,...,0.4,1.0,1.0,1.0,0,8.14350,20141201T000000,1,1,1
1,1077430,1314167,2500,2500,2500,60 months,15.27,59.83,C,C4,...,0.8,1.0,1.0,1.0,1,2.39320,20161201T000000,1,1,1
2,1077175,1313524,2400,2400,2400,36 months,15.96,84.33,C,C5,...,1.0,1.0,1.0,1.0,0,8.25955,20141201T000000,1,1,1
3,1076863,1277178,10000,10000,10000,36 months,13.49,339.31,C,C1,...,0.2,1.0,1.0,1.0,0,8.27585,20141201T000000,0,1,1
4,1075269,1311441,5000,5000,5000,36 months,7.90,156.46,A,A4,...,0.8,1.0,1.0,1.0,0,5.21533,20141201T000000,1,1,1


In [4]:
loans['safe_loans'] = loans['bad_loans'].apply(lambda x : +1 if x==0 else -1)
loans = loans.drop(['bad_loans'], axis = 1)
loans.shape

(122607, 68)

In [5]:
# We only consider the following amount of features

features = ['grade',              # grade of the loan
            'term',               # the term of the loan
            'home_ownership',     # home_ownership status: own, mortgage or rent
            'emp_length',         # number of years of employment
           ]
target = 'safe_loans'

In [6]:
# Extract the feature columns and target column
loans = loans[features + [target]]

In [7]:
loans.shape

(122607, 5)

## Subsample dataset to make sure classes are balanced

In [8]:
import json
with open('train-idx.json', 'r') as f:
    train_idx = json.load(f)
with open('validation-idx.json', 'r') as f:
    validation_idx = json.load(f)

In [9]:
print(len(train_idx))
print(len(validation_idx))
print(len(train_idx) + len(validation_idx))

37224
9284
46508


# Encoding category data

In [10]:
loans.head(5)
print(loans.shape)
print(len(loans))

(122607, 5)
122607


In [11]:
list(loans.columns.values)

['grade', 'term', 'home_ownership', 'emp_length', 'safe_loans']

In [12]:
def encoding_binary_feature(data, y_label):
    labels =  data.select_dtypes(include=[object])
    encoded_features =[]
    #encoded_values = np.transpose(np.array(np.ones(len(data),)))
    encoded_values = pd.DataFrame(data[y_label])   
    
    for label in labels:
        
        distinct_features = list(set(data[label].values))
    
        for feature in distinct_features:
            encoded_features.append(str(label + '.'+feature))
            
            #new_array = np.array(np.ones(len(data),))
            encoded_values[encoded_features[-1]] = data[label].apply(lambda x : +1 if x==feature else 0)
    
    return encoded_values

In [13]:
loans_data = encoding_binary_feature(loans,'safe_loans')
print(loans_data.columns.values)

['safe_loans' 'grade.D' 'grade.F' 'grade.A' 'grade.C' 'grade.E' 'grade.B'
 'grade.G' 'term. 36 months' 'term. 60 months' 'home_ownership.OTHER'
 'home_ownership.MORTGAGE' 'home_ownership.OWN' 'home_ownership.RENT'
 'emp_length.6 years' 'emp_length.5 years' 'emp_length.4 years'
 'emp_length.9 years' 'emp_length.< 1 year' 'emp_length.n/a'
 'emp_length.3 years' 'emp_length.8 years' 'emp_length.2 years'
 'emp_length.10+ years' 'emp_length.1 year' 'emp_length.7 years']


In [25]:
#from sklearn.preprocessing import LabelEncoder, OneHotEncoder
#loans_object = loans.select_dtypes(include=[object])

#le = LabelEncoder()

#loans_object = loans_object.apply(le.fit_transform)
#loans_object.head()

#onehotencoder = OneHotEncoder()
#X_features = onehotencoder.fit_transform(loans_object).toarray()
#print(loans_object.shape)

#Y_loans = loans.select_dtypes(exclude=[object])
#loans_transform =  pd.concat([loans_left, pd.DataFrame(loans_object)], axis=1,join='inner') 
#print(Y_loans.shape)

In [14]:
print(loans_data.shape)

(122607, 26)


## Train-test split

We split the data into a train test split with 80% of the data in the training set and 20% of the data in the test set. We use `seed=1` so that everyone gets the same result.

In [15]:
train_data = loans_data.iloc[train_idx].drop(['safe_loans'], axis = 1)
train_Y = loans_data['safe_loans'].iloc[train_idx]

In [16]:
test_data = loans_data.iloc[validation_idx].drop(['safe_loans'], axis = 1)
test_Y = loans_data['safe_loans'].iloc[validation_idx]

In [17]:
safe_loans = (train_Y == 1).sum() + (test_Y ==1).sum()
bad_loans = (train_Y == -1).sum() + (test_Y == -1).sum()
percentage = (safe_loans/(safe_loans + bad_loans)*100)
print('Safe loan percentage: ', percentage)

Safe loan percentage:  50.2236174422


# Decision tree implementation

In this section, we will implement binary decision trees from scratch. There are several steps involved in building a decision tree. For that reason, we have split the entire assignment into several sections.

## Function to count number of mistakes while predicting majority class

Recall from the lecture that prediction at an intermediate node works by predicting the **majority class** for all data points that belong to this node.

Now, we will write a function that calculates the number of **missclassified examples** when predicting the **majority class**. This will be used to help determine which feature is the best to split on at a given node of the tree.

**Note**: Keep in mind that in order to compute the number of mistakes for a majority classifier, we only need the label (y values) of the data points in the node. 

** Steps to follow **:
* ** Step 1:** Calculate the number of safe loans and risky loans.
* ** Step 2:** Since we are assuming majority class prediction, all the data points that are **not** in the majority class are considered **mistakes**.
* ** Step 3:** Return the number of **mistakes**.


In [18]:
def intermediate_node_num_mistakes(labels_in_node, key_value):
    # Corner case: If labels_in_node is empty, return 0
    if len(labels_in_node) == 0:
        return 0
    
    # Count the number of 1's (safe loans)
    #num_safe_loans = (labels_in_node==1).sum()
    
    # Count the number of -1's (risky loans)
    #num_risky_loans = (labels_in_node ==-1).sum()
                
    # Return the number of mistakes that the majority classifier makes.
    #return num_safe_loans if num_safe_loans < num_risky_loans else num_risky_loans
    
    if(key_value == -1):
        return (labels_in_node==1).sum()
    elif key_value == 1:
        return (labels_in_node==-1).sum()
    else:
        num_safe_loans = (labels_in_node==1).sum()
    
        # Count the number of -1's (risky loans)
        num_risky_loans = (labels_in_node ==-1).sum()
                
        # Return the number of mistakes that the majority classifier makes.
        return num_safe_loans if num_safe_loans < num_risky_loans else num_risky_loans

Because there are several steps in this assignment, we have introduced some stopping points where you can check your code and make sure it is correct before proceeding. To test your `intermediate_node_num_mistakes` function, run the following code until you get a **Test passed!**, then you should proceed. Otherwise, you should spend some time figuring out where things went wrong.

In [19]:
# Test case 1
example_labels = np.array([-1, -1, 1, 1, 1])
if intermediate_node_num_mistakes(example_labels, 0) == 2:
    print ('Test passed!')
else:
    print ('Test 1 failed... try again!')

# Test case 2
example_labels = np.array([-1, -1, 1, 1, 1, 1, 1])
if intermediate_node_num_mistakes(example_labels, 0) == 2:
    print ('Test passed!')
else:
    print ('Test 2 failed... try again!')
    
# Test case 3
example_labels = np.array([-1, -1, -1, -1, -1, 1, 1])
if intermediate_node_num_mistakes(example_labels, 0) == 2:
    print ('Test passed!')
else:
    print ('Test 3 failed... try again!')

Test passed!
Test passed!
Test passed!


## Function to pick best feature to split on

The function **best_splitting_feature** takes 3 arguments: 
1. The data (includes all of the feature columns and label column)
2. The features to consider for splits (a list of strings of column names to consider for splits)
3. The name of the target/label column (string)

The function will loop through the list of possible features, and consider splitting on each of them. It will calculate the classification error of each split and return the feature that had the smallest classification error when split on.

Recall that the **classification error** is defined as follows:
$$
\mbox{classification error} = \frac{\mbox{# mistakes}}{\mbox{# total examples}}
$$

Follow these steps: 
* **Step 1:** Loop over each feature in the feature list
* **Step 2:** Within the loop, split the data into two groups: one group where all of the data has feature value 0 or False (we will call this the **left** split), and one group where all of the data has feature value 1 or True (we will call this the **right** split). Make sure the **left** split corresponds with 0 and the **right** split corresponds with 1 to ensure your implementation fits with our implementation of the tree building process.
* **Step 3:** Calculate the number of misclassified examples in both groups of data and use the above formula to compute the **classification error**.
* **Step 4:** If the computed error is smaller than the best error found so far, store this **feature and its error**.

This may seem like a lot, but we have provided pseudocode in the comments in order to help you implement the function correctly.

**Note:** Remember that since we are only dealing with binary features, we do not have to consider thresholds for real-valued features. This makes the implementation of this function much easier.

In [20]:
def best_splitting_feature(data, features, target):
    
    best_feature = None # Keep track of the best feature 
    best_error = 10     # Keep track of the best error so far 
    # Note: Since error is always <= 1, we should intialize it with something larger than 1.

    # Convert to float to make sure error gets computed correctly.
    num_data_points = float(len(target))  
    
    # Loop through each feature to consider splitting on that feature
    for feature in features:
        
        # The left split will have all data points where the feature value is 0
        left_split = data[data[feature] == 0]
        
        # The right split will have all data points where the feature value is 1
        right_split =  data[data[feature] == 1]
            
        # Calculate the number of misclassified examples in the left split.
        # Remember that we implemented a function for this! (It was called intermediate_node_num_mistakes)
        left_mistakes =  intermediate_node_num_mistakes(target[data[feature] == 0], -1)           

        # Calculate the number of misclassified examples in the right split.
        right_mistakes = intermediate_node_num_mistakes(target[data[feature] == 1], 1)      
            
        # Compute the classification error of this split.
        # Error = (# of mistakes (left) + # of mistakes (right)) / (# of data points)
        error = (left_mistakes + right_mistakes) / num_data_points
        #print(str(error) + feature)
        # If this is the best error we have found so far, store the feature as best_feature and the error as best_error
        if error <best_error:
            best_error = error
            best_feature = feature
        
    
    return best_feature # Return the best feature we found

To test your `best_splitting_feature` function, run the following code:

In [21]:
print(train_data[train_data['grade.A'] == 1].shape)
print((train_data['term. 60 months'] == 1).sum())
print((train_data['term. 36 months'] == 1).sum())
print((train_data['grade.C'] == 1).sum())

(5130, 25)
9223
28001
9412


In [22]:
features = train_data.columns.values
features

array(['grade.D', 'grade.F', 'grade.A', 'grade.C', 'grade.E', 'grade.B',
       'grade.G', 'term. 36 months', 'term. 60 months',
       'home_ownership.OTHER', 'home_ownership.MORTGAGE',
       'home_ownership.OWN', 'home_ownership.RENT', 'emp_length.6 years',
       'emp_length.5 years', 'emp_length.4 years', 'emp_length.9 years',
       'emp_length.< 1 year', 'emp_length.n/a', 'emp_length.3 years',
       'emp_length.8 years', 'emp_length.2 years', 'emp_length.10+ years',
       'emp_length.1 year', 'emp_length.7 years'], dtype=object)

In [23]:
train_Y.head(1)

1   -1
Name: safe_loans, dtype: int64

In [24]:
train_data.head(2)

,grade.D,grade.F,grade.A,grade.C,grade.E,grade.B,grade.G,term. 36 months,term. 60 months,home_ownership.OTHER,...,emp_length.4 years,emp_length.9 years,emp_length.< 1 year,emp_length.n/a,emp_length.3 years,emp_length.8 years,emp_length.2 years,emp_length.10+ years,emp_length.1 year,emp_length.7 years
1,0,0,0,1,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
6,0,1,0,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0


In [25]:
returned_feature = best_splitting_feature(train_data, features, train_Y)
if  returned_feature == 'term. 36 months':
    print ('Test passed!')
else:
    print ('Test failed... try again!')
    print('Returned feature: ', returned_feature)

Test passed!


## Building the tree

With the above functions implemented correctly, we are now ready to build our decision tree. Each node in the decision tree is represented as a dictionary which contains the following keys and possible values:

    { 
       'is_leaf'            : True/False.
       'prediction'         : Prediction at the leaf node.
       'left'               : (dictionary corresponding to the left tree).
       'right'              : (dictionary corresponding to the right tree).
       'splitting_feature'  : The feature that this node splits on.
    }

First, we will write a function that creates a leaf node given a set of target values. Fill in the places where you find `## YOUR CODE HERE`. There are **three** places in this function for you to fill in.

In [26]:
def create_leaf(target_values):
    
    # Create a leaf node
    leaf = {'splitting_feature' : None,
            'left' : None,
            'right' : None,
            'is_leaf': True    }   
    
    # Count the number of data points that are +1 and -1 in this node.
    num_ones = len(target_values[target_values == +1])
    num_minus_ones = len(target_values[target_values == -1])
    
    # For the leaf node, set the prediction to be the majority class.
    # Store the predicted class (1 or -1) in leaf['prediction']
    if num_ones > num_minus_ones:
        leaf['prediction'] =          1
    else:
        leaf['prediction'] =          -1
        
    # Return the leaf node        
    return leaf 

We have provided a function that learns the decision tree recursively and implements 3 stopping conditions:
1. **Stopping condition 1:** All data points in a node are from the same class.
2. **Stopping condition 2:** No more features to split on.
3. **Additional stopping condition:** In addition to the above two stopping conditions covered in lecture, in this assignment we will also consider a stopping condition based on the **max_depth** of the tree. By not letting the tree grow too deep, we will save computational effort in the learning process. 


In [27]:
def decision_tree_create(data, features, target, current_depth = 0, max_depth = 10):
    
    remaining_features = features[:] # Make a copy of the features.
    
    target_values = target
    print ("--------------------------------------------------------------------")
    print ("Subtree, depth = %s (%s data points)." % (current_depth, len(target_values)))
    

    # Stopping condition 1
    # (Check if there are mistakes at current node.
    # Recall you wrote a function intermediate_node_num_mistakes to compute this.)
    if intermediate_node_num_mistakes(target,0) == 0: 
        print ("Stopping condition 1 reached." )    
        # If not mistakes at current node, make current node a leaf node
        return create_leaf(target_values)
    
    # Stopping condition 2 (check if there are remaining features to consider splitting on)
    if len(remaining_features) == 0: 
        print ("Stopping condition 2 reached.")    
        # If there are no remaining features to consider, make current node a leaf node
        return create_leaf(target_values)    
    
    # Additional stopping condition (limit tree depth)
    if current_depth >= max_depth: 
        print ("Reached maximum depth. Stopping for now.")
        # If the max tree depth has been reached, make current node a leaf node
        return create_leaf(target_values)

    # Find the best splitting feature (recall the function best_splitting_feature implemented above)

    splitting_feature = best_splitting_feature(data, features, target_values)
    # Split on the best feature that we found. 
    left_split = data[data[splitting_feature] == 0]
    left_target = target_values[data[splitting_feature] == 0]
    
    right_split = data[data[splitting_feature] == 1] 
    right_target = target_values[data[splitting_feature] == 1]
    
    #remaining_features =  remaining_features.remove(splitting_feature)
    remaining_features = [x for x in remaining_features if x != splitting_feature]
    
    print ("Split on feature %s. (%s, %s)" % (\
                      splitting_feature, len(left_split), len(right_split)))
    
    # Create a leaf node if the split is "perfect"
    if len(left_split) == len(data):
        print ("Creating leaf node.")
        return create_leaf(left_target)
    if len(right_split) == len(data):
        print ("Creating leaf node.")
        return create_leaf(right_target)

        
    # Repeat (recurse) on left and right subtrees
    left_tree = decision_tree_create(left_split, remaining_features, left_target, current_depth + 1, max_depth)        

    right_tree = decision_tree_create(right_split, remaining_features, right_target, current_depth + 1, max_depth) 

    return {'is_leaf'          : False, 
            'prediction'       : None,
            'splitting_feature': splitting_feature,
            'left'             : left_tree, 
            'right'            : right_tree}

Here is a recursive function to count the nodes in your tree:

In [28]:
def count_nodes(tree):
    if tree['is_leaf']:
        return 1
    return 1 + count_nodes(tree['left']) + count_nodes(tree['right'])

Run the following test code to check your implementation. Make sure you get **'Test passed'** before proceeding.

In [29]:
small_data_decision_tree = decision_tree_create(train_data, features, train_Y, max_depth = 3)

if count_nodes(small_data_decision_tree) == 13:
    print ('Test passed!')
else:
    print ('Test failed... try again!')
    print ('Number of nodes found                :', count_nodes(small_data_decision_tree))
    print ('Number of nodes that should be there : 13' )

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature term. 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Split on feature grade.A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Split on feature home_ownership.OTHER. (9119, 3)
--------------------------------------------------------------------
Subtree, depth = 3 (9119 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------------------------
Subtree, depth = 3 (3 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------------------------
Subtree, depth = 2 (101 data points).
Split on feature term. 60 months. (0, 101)
Creating leaf node.
--------------------------------------------------------------------
Subtre

## Build the tree!

Now that all the tests are passing, we will train a tree model on the **train_data**. Limit the depth to 6 (**max_depth = 6**) to make sure the algorithm doesn't run for too long. Call this tree **my_decision_tree**. 

**Warning**: This code block may take 1-2 minutes to learn. 

In [30]:
# Make sure to cap the depth at 6 by using max_depth = 6
my_decision_tree =decision_tree_create(train_data, features, train_Y, max_depth = 6)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature term. 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Split on feature grade.A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Split on feature home_ownership.OTHER. (9119, 3)
--------------------------------------------------------------------
Subtree, depth = 3 (9119 data points).
Split on feature emp_length.n/a. (8898, 221)
--------------------------------------------------------------------
Subtree, depth = 4 (8898 data points).
Split on feature grade.B. (7884, 1014)
--------------------------------------------------------------------
Subtree, depth = 5 (7884 data points).
Split on feature emp_length.8 years. (7507, 377)
--------------------------------------------------------------------
Subtree, depth

Split on feature grade.D. (186, 0)
Creating leaf node.
--------------------------------------------------------------------
Subtree, depth = 4 (977 data points).
Split on feature grade.D. (977, 0)
Creating leaf node.


## Making predictions with a decision tree

As discussed in the lecture, we can make predictions from the decision tree with a simple recursive function. Below, we call this function `classify`, which takes in a learned `tree` and a test point `x` to classify.  We include an option `annotate` that describes the prediction path when set to `True`.

Fill in the places where you find `## YOUR CODE HERE`. There is **one** place in this function for you to fill in.

In [44]:
def classify(tree, x, annotate = False):   
    # if the node is a leaf node.
    if tree['is_leaf']:
        if annotate: 
            print ("At leaf, predicting %s" % tree['prediction'])
        return tree['prediction'] 
    else:
        # split on feature.
        split_feature_value = x[tree['splitting_feature']]
        if annotate: 
            print ("Split on %s = %s" % (tree['splitting_feature'], split_feature_value))
        if split_feature_value == 0:
            return classify(tree['left'], x, annotate)
        else:
            return classify(tree['right'], x, annotate)

Now, let's consider the first example of the test set and see what `my_decision_tree` model predicts for this data point.

In [32]:
test_data.iloc[0]

grade.D                    1
grade.F                    0
grade.A                    0
grade.C                    0
grade.E                    0
grade.B                    0
grade.G                    0
term. 36 months            0
term. 60 months            1
home_ownership.OTHER       0
home_ownership.MORTGAGE    0
home_ownership.OWN         0
home_ownership.RENT        1
emp_length.6 years         0
emp_length.5 years         0
emp_length.4 years         0
emp_length.9 years         0
emp_length.< 1 year        0
emp_length.n/a             0
emp_length.3 years         0
emp_length.8 years         0
emp_length.2 years         1
emp_length.10+ years       0
emp_length.1 year          0
emp_length.7 years         0
Name: 24, dtype: int64

In [33]:
print ('Predicted class: %s ' % classify(my_decision_tree, test_data.iloc[0]))

Predicted class: -1 


## Evaluating your decision tree

Now, we will write a function to evaluate a decision tree by computing the classification error of the tree on the given dataset.

Again, recall that the **classification error** is defined as follows:
$$
\mbox{classification error} = \frac{\mbox{# mistakes}}{\mbox{# total examples}}
$$

Now, write a function called `evaluate_classification_error` that takes in as input:
1. `tree` (as described above)
2. `data` (an SFrame)
3. `target` (a string - the name of the target/label column)

This function should calculate a prediction (class label) for each row in `data` using the decision `tree` and return the classification error computed using the above formula. Fill in the places where you find `## YOUR CODE HERE`.

In [34]:
def evaluate_classification_error(tree, data, target):
    # Apply the classify(tree, x) to each row in your data
    #prediction = data.apply(lambda x: classify(tree, x))
    prediction =[]
    for i in range(len(data)):
        prediction.append(classify(tree, data.iloc[i]))
    
    # Once you've made the predictions, calculate the classification error and return it
    errors = (prediction == target).sum()/len(target)
    return errors
    

In [35]:
evaluate_classification_error(my_decision_tree, test_data, test_Y)

0.61805256355019389

In [36]:
evaluate_classification_error(my_decision_tree, train_data, train_Y)

0.61538254889318722

**Quiz Question:** Rounded to 2nd decimal point, what is the classification error of **my_decision_tree** on the **test_data**?

## Printing out a decision stump

As discussed in the lecture, we can print out a single decision stump (printing out the entire tree is left as an exercise to the curious reader). 

In [37]:
def print_stump(tree, name = 'root'):
    split_name = tree['splitting_feature'] # split_name is something like 'term. 36 months'
    if split_name is None:
        print ("(leaf, label: %s)" % tree['prediction'])
        return None
    split_feature, split_value = split_name.split('.')
    print ('                       %s' % name)
    print ('         |---------------|----------------|')
    print ('         |                                |')
    print ('         |                                |')
    print ('         |                                |')
    print ('  [{0} == 0]               [{0} == 1]    '.format(split_name))
    print ('         |                                |')
    print ('         |                                |')
    print ('         |                                |')
    print ('    (%s)                         (%s)' \
        % (('leaf, label: ' + str(tree['left']['prediction']) if tree['left']['is_leaf'] else 'subtree'),
           ('leaf, label: ' + str(tree['right']['prediction']) if tree['right']['is_leaf'] else 'subtree')))

In [38]:
print_stump(my_decision_tree)

                       root
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [term. 36 months == 0]               [term. 36 months == 1]    
         |                                |
         |                                |
         |                                |
    (subtree)                         (subtree)


**Quiz Question:** What is the feature that is used for the split at the root node?

### Exploring the intermediate left subtree

The tree is a recursive dictionary, so we do have access to all the nodes! We can use
* `my_decision_tree['left']` to go left
* `my_decision_tree['right']` to go right

In [39]:
print_stump(my_decision_tree['left'], my_decision_tree['splitting_feature'])

                       term. 36 months
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [grade.A == 0]               [grade.A == 1]    
         |                                |
         |                                |
         |                                |
    (subtree)                         (leaf, label: 1)


In [40]:
print_stump(my_decision_tree['right'], my_decision_tree['splitting_feature'])

                       term. 36 months
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [grade.A == 0]               [grade.A == 1]    
         |                                |
         |                                |
         |                                |
    (subtree)                         (subtree)


### Exploring the left subtree of the left subtree


In [41]:
print_stump(my_decision_tree['left']['left'], my_decision_tree['left']['splitting_feature'])

                       grade.A
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [home_ownership.OTHER == 0]               [home_ownership.OTHER == 1]    
         |                                |
         |                                |
         |                                |
    (subtree)                         (subtree)


In [42]:
print_stump(my_decision_tree['left']['left']['left'], my_decision_tree['left']['splitting_feature'])

                       grade.A
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [emp_length.n/a == 0]               [emp_length.n/a == 1]    
         |                                |
         |                                |
         |                                |
    (subtree)                         (subtree)


In [43]:
print_stump(my_decision_tree['left']['right'], my_decision_tree['left']['splitting_feature'])

(leaf, label: 1)
